In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from typing import Tuple, Any
from isaacgym import gymapi, gymtorch
from nerf_grasping.sim import ig_utils
from nerf_grasping.sim import ig_objects

import time
import torch
import numpy as np

Importing module 'gym_37' (/scr-ssd/ksrini/Downloads/isaacgym-3/python/isaacgym/_bindings/linux-x86_64/gym_37.so)
Setting GYM_USD_PLUG_INFO_PATH to /scr-ssd/ksrini/Downloads/isaacgym-3/python/isaacgym/_bindings/linux-x86_64/usd/plugInfo.json
PyTorch version 1.11.0+cu113
Device count 1
/scr-ssd/ksrini/Downloads/isaacgym-3/python/isaacgym/_bindings/src/gymtorch
Using /afs/cs.stanford.edu/u/ksrini/.cache/torch_extensions/py37_cu113 as PyTorch extensions root...
Emitting ninja build file /afs/cs.stanford.edu/u/ksrini/.cache/torch_extensions/py37_cu113/gymtorch/build.ninja...
Building extension module gymtorch...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)
Loading extension module gymtorch...


In [3]:
import os
from pathlib import Path

root_dir = Path(os.path.abspath("./")).parents[0]
asset_dir = f"{root_dir}/assets"

# Define Robot and Env

## Define Robot

In [71]:
class FingertipRobot:
    """Robot controlling 3 (or n) spheres as "fingers" to grasp an lift an object"""

    def __init__(
        self,
        gym: gymapi.Gym,
        sim: Any,
        env: Any,
        target_height: float = 0.07,
        use_grad_est: bool = True,
        grasp_vars: Tuple[torch.Tensor, torch.Tensor] = None,
        **kwargs,
    ):
        """
        Initializes Fingertip-only point mass robot
        """
        self.gym = gym
        self.sim = sim
        self.env = env
        self.target_height = target_height
        self._use_grad_est = use_grad_est
        self.grasp_points, self.grasp_normals = grasp_vars
        self.actors = self.create_spheres()

    def create_spheres(self):
        asset_options = gymapi.AssetOptions()
        asset_options.fix_base_link = False
        asset_options.angular_damping = 0.0
        asset_options.max_angular_velocity = 0.0
        asset_options.slices_per_cylinder = 40
        markers = []
        actors = []
        colors = np.array([[0, 0, 1], [0, 1, 0], [1, 0, 0]], dtype="float32")
        ftip_start_pos = grasp_points - grasp_normals * 0.1
        for i, pos in enumerate(ftip_start_pos):
            color = colors[i]
            pose = gymapi.Transform()
            pose.p.x = pos[0]
            pose.p.y = pos[1]
            pose.p.z = pos[2]
            marker_asset = gym.create_sphere(self.sim, 0.005, asset_options)
            markers.append(marker_asset)
            actor_handle = gym.create_actor(
                env, marker_asset, pose, f"fingertip_{i}",
            )
            actors.append(actor_handle)
            gym.set_rigid_body_color(
                env,
                actor_handle,
                0,
                gymapi.MESH_VISUAL,
                gymapi.Vec3(*color),
            )
        return actors

    def setup_tensors(self):
        _rb_states = self.gym.acquire_rigid_body_state_tensor(self.sim)
        # (num_rigid_bodies, 13)
        rb_count = self.gym.get_actor_rigid_body_count(self.env, self.actors[0])
        rb_start_index = self.gym.get_actor_rigid_body_index(env, self.actors[0], 0, gymapi.DOMAIN_SIM)

        # NOTE: simple indexing will return a view of the data but advanced indexing will return a copy breaking the updateing
        self.rb_states = gymtorch.wrap_tensor(_rb_states)[rb_start_index : rb_start_index + rb_count*len(self.actors), :]

    def get_position(self):
        """Returns tip positions"""
        return self.rb_states[:, 0:3]

    def get_velocity(self):
        """Returns linear velocity"""
        return self.rb_states[:, 7:10]
    
    def apply_fingertip_forces(self, global_fingertip_forces):
        """Applies forces to individual actors"""
        assert global_fingertip_forces.shape == (3, 3)
        for f, actor_handle in zip(global_fingertip_forces, self.actors):
            rb_handle = gym.get_actor_rigid_body_handle(env, actor_handl, 0)
            fx, fy, fz = f
            self.gym.apply_body_forces(
                self.env,
                rb_handle,
                gymapi.Vec3(fx, fy, fz),      # force
                gymapi.Vec3(0.0, 0.0, 0.0),   # torque
                gymapi.CoordinateSpace.GLOBAL_SPACE,
            )
        return

    def reset_actor(self, grasp_vars=None):
        if grasp_vars is not None:
            self.grasp_points, self.grasp_normals = grasp_vars
        ftip_start_pos = self.grasp_points - self.grasp_normals * 0.05
        for pos, handle in zip(ftip_start_pos, self.actors):
            state = self.gym.get_actor_rigid_body_states(
                self.env, handle, gymapi.STATE_POS
            )
            state["pose"]["p"].fill(tuple(pos))
            assert self.gym.set_actor_rigid_body_states(
                self.env, handle, state, gymapi.STATE_POS
            ), "gym.set_actor_rigid_body_states failed"
            state = self.gym.get_actor_rigid_body_states(
                self.env, handle, gymapi.STATE_VEL
            )
            state["vel"]["linear"].fill((0.0, 0.0, 0.0))
            state["vel"]["angular"].fill((0.0, 0.0, 0.0))
            assert self.gym.set_actor_rigid_body_states(
                self.env, handle, state, gymapi.STATE_VEL
            ), "gym.set_actor_rigid_body_states failed"
        return

    @property
    def position(self):
        return self.get_position()
    
    @propety
    def velocity(self):
        return self.get_velocity()
    
    @property
    def use_grad_est(self):
        return self._use_grad_est

    @use_grad_est.setter
    def use_grad_est(self, x):
        self._use_grad_est = x

    @property
    def use_true_normals(self):
        return not self._use_grad_est

NameError: name 'propety' is not defined

In [40]:
[gym.get_actor_rigid_body_handle(env, k, 0) for k in robot.actors]

[1, 2, 3]

## IG helpers

In [5]:
def setup_viewer():
    viewer = gym.create_viewer(sim, gymapi.CameraProperties())
    # position outside stage
    cam_pos = gymapi.Vec3(0.7, 0.175, 0.6)
    # position above banana
    cam_pos = gymapi.Vec3(0.1, 0.02, 0.4)
    cam_target = gymapi.Vec3(0, 0, 0.2)
    gym.viewer_camera_look_at(viewer, env, cam_pos, cam_target)
    return viewer


def step_gym():
    gym.simulate(sim)
    gym.fetch_results(sim, True)

    gym.step_graphics(sim)
    if viewer is not None:
        gym.draw_viewer(viewer, sim, True)
        gym.sync_frame_time(sim)
    refresh_tensors()


def setup_env():
    plane_params = gymapi.PlaneParams()
    plane_params.normal = gymapi.Vec3(0, 0, 1)  # z-up!
    gym.add_ground(sim, plane_params)

    spacing = 1.0
    env_lower = gymapi.Vec3(-spacing, -spacing, 0.0)
    env_upper = gymapi.Vec3(spacing, spacing, spacing)
    env = gym.create_env(sim, env_lower, env_upper, 0)
    return env


def refresh_tensors():
    gym.refresh_mass_matrix_tensors(sim)
    gym.refresh_jacobian_tensors(sim)
    gym.refresh_dof_state_tensor(sim)
    gym.refresh_rigid_body_state_tensor(sim)


def setup_sim():
    args = ig_utils.parse_arguments(description="Trifinger test")
    # only tested with this one
    assert args.physics_engine == gymapi.SIM_PHYSX

    # configure sim
    sim_params = gymapi.SimParams()
    sim_params.dt = 1.0 / 60.0

    sim_params.up_axis = gymapi.UP_AXIS_Z
    sim_params.gravity = gymapi.Vec3(0.0, 0.0, -9.8)

    sim_params.physx.solver_type = 1
    sim_params.physx.num_position_iterations = 6
    sim_params.physx.num_velocity_iterations = 0
    sim_params.physx.num_threads = args.num_threads
    sim_params.physx.use_gpu = args.use_gpu
    # sim_params.physx.use_gpu = True

    # sim_params.use_gpu_pipeline = True
    sim_params.use_gpu_pipeline = False
    sim = gym.create_sim(
        args.compute_device_id,
        args.graphics_device_id,
        args.physics_engine,
        sim_params,
    )
    assert sim is not None

    # intensity = 0.01 # for nerf generation
    # ambient = 0.21 / intensity
    intensity = 0.5
    ambient = 0.10 / intensity
    intensity = gymapi.Vec3(intensity, intensity, intensity)
    ambient = gymapi.Vec3(ambient, ambient, ambient)

    gym.set_light_parameters(sim, 0, intensity, ambient, gymapi.Vec3(0.5, 1, 1))
    gym.set_light_parameters(sim, 1, intensity, ambient, gymapi.Vec3(1, 0, 1))
    gym.set_light_parameters(sim, 2, intensity, ambient, gymapi.Vec3(0.5, -1, 1))
    gym.set_light_parameters(sim, 3, intensity, ambient, gymapi.Vec3(0, 0, 1))
    return sim


def setup_stage(env):
    # this one is convex decomposed
    stage_urdf_file = "trifinger/robot_properties_fingers/urdf/high_table_boundary.urdf"
    # stage_urdf_file = "trifinger/robot_properties_fingers/urdf/trifinger_stage.urdf"
    # stage_urdf_file = "trifinger/robot_properties_fingers/urdf/stage.urdf"

    asset_options = gymapi.AssetOptions()
    asset_options.disable_gravity = True
    asset_options.fix_base_link = True
    asset_options.flip_visual_attachments = False
    asset_options.use_mesh_materials = True
    asset_options.thickness = 0.001

    stage_asset = gym.load_asset(sim, asset_dir, stage_urdf_file, asset_options)
    gym.create_actor(
        env, stage_asset, gymapi.Transform(), "Stage", 0, 0, segmentationId=1
    )

## Setup

In [68]:
def position_control(observation, desired_position):
    """Computes joint torques using tip link jacobian to achieve desired tip position"""
    tip_velocities = observation[:, 7:10]
    tip_positions = observation[:, :3]
    assert tip_positions.shape == desired_position.shape
    xyz_force = -2 * (tip_positions - desired_position) - 1. * tip_velocities
    return xyz_force

In [6]:
gym = gymapi.acquire_gym()
sim = setup_sim()
env = setup_env()
setup_stage(env)
viewer = setup_viewer()
grasp_points = torch.tensor(
    [
        [-0.00693, 0.085422, 0.013867],
        [0.018317, -0.001611, 0.013867],
        [-0.058538, -0.051027, 0.013867],
    ]
)

grasp_normals = torch.tensor([[1, -1.5, 0.0], [-2, 1.0, 0.0], [1, 0.0, 0.0]])
grasp_normals = grasp_normals / grasp_normals.norm(dim=1, keepdim=True)
grasp_vars = (grasp_points, grasp_normals)
robot = FingertipRobot(gym, sim, env, grasp_vars=grasp_vars)
Obj = ig_objects.Banana
obj = Obj(gym, sim, env)
robot.setup_tensors()
obj.setup_tensors()
for i in range(4): step_gym()

In [ ]:
def random_forces(timestep):
    fx = -np.sin(timestep * np.pi / 10) * 0.025 + 0.001
    fy = -np.sin(timestep * np.pi / 5) * 0.025 + 0.001
    f = np.array([[fx, fy, 0.]]*3)
    return f

In [43]:
robot.reset_actor()
obj.reset_actor()

In [70]:
robot.reset_actor()
obj.reset_actor()
for i in range(1000):
    time.sleep(0.05)
    step_gym()
    f = position_control(robot.rb_states[:], grasp_points) * .1
    if i % 10 == 0:
        print("POSITION:", robot.get_pos())
        # print("VELOCITY:", robot.get_vel())
        print("FORCE MAGNITUDE:", f.norm())
    robot.apply_fingertip_forces(f)

POSITION: tensor([[-0.0342,  0.1264,  0.0120],
        [ 0.0623, -0.0236,  0.0120],
        [-0.1078, -0.0510,  0.0120]])
FORCE MAGNITUDE: tensor(0.0274)
POSITION: tensor([[-0.0827,  0.1991,  0.0070],
        [ 0.1405, -0.0627,  0.0070],
        [-0.1951, -0.0510,  0.0070]])
FORCE MAGNITUDE: tensor(2.4512)
POSITION: tensor([[-0.0695,  0.1997,  0.0157],
        [ 0.1925, -0.0862,  0.0156],
        [-0.2024, -0.0575,  0.0157]])
FORCE MAGNITUDE: tensor(0.0537)
POSITION: tensor([[-0.0691,  0.1999,  0.0157],
        [ 0.1926, -0.0858,  0.0156],
        [-0.2020, -0.0591,  0.0157]])
FORCE MAGNITUDE: tensor(0.0537)
POSITION: tensor([[-0.0687,  0.2001,  0.0157],
        [ 0.1928, -0.0854,  0.0156],
        [-0.2016, -0.0606,  0.0157]])
FORCE MAGNITUDE: tensor(0.0537)
POSITION: tensor([[-0.0683,  0.2002,  0.0157],
        [ 0.1930, -0.0851,  0.0156],
        [-0.2013, -0.0620,  0.0156]])
FORCE MAGNITUDE: tensor(0.0536)
POSITION: tensor([[-0.0680,  0.2004,  0.0157],
        [ 0.1931, -0.0847,  0

KeyboardInterrupt: 

In [25]:
robot.reset_actor(grasp_vars)

In [ ]:
gym.get_actor_dof_count(env, 3)

In [35]:
for i in range(4):
    print(gym.get_actor_name(env, i))
    print(gym.get_sim_rigid_body_states(sim, gymapi.STATE_POS)[i]["pose"]["p"])

Stage
(0., 0., 0.)
fingertip_0
(-2.0689284e-07, 0.14587425, 0.09843944)
fingertip_1
(0.03, -0.15, 0.007)
fingertip_2
(-0.03, -0.15, 0.007)


In [ ]:
Obj = ig_objects.Banana
obj = Obj(gym, sim, env)
robot.setup_tensors()
obj.setup_tensors()

In [ ]:
gym.get_actor_actuator_count(env, 4)

In [ ]:
robot.setup_tensors()

In [ ]:
for i in range(5):
    print(gym.get_actor_name(env, i))

## Load env and object

In [ ]:
Obj = ig_objects.Banana
grasp_points = torch.tensor(
    [[0.0, 0.05, 0.05], [0.03, -0.05, 0.05], [-0.03, -0.05, 0.05]]
)
grasp_normals = torch.tensor([[0.0, -1.0, 0.0], [0.0, 1.0, 0.0], [0.0, 1.0, 0.0]])
robot_kwargs = dict(grasp_vars=(grasp_points, grasp_normals))

tf = TriFingertipEnv(viewer=True, robot=True, Obj=Obj, **robot_kwargs)